In [0]:
%tensorflow_version 2.x
import os
import csv
import tensorflow as tf
from tensorflow import keras
#import keras
import matplotlib.pyplot as plt
#import cv2
#from keras.models import Model
from tensorflow.keras import Model
import numpy as np
import tensorflow_addons as tfa
import math
tf.__version__

'2.2.0-rc3'

In [0]:
!pip install -U -q kaggle
!mkdir /content/.kaggle
from google.colab import files
files.upload()
os.path.dirname('.kaggle')

In [0]:
import json
token = {"username":"shreegowri","key":"f70a15b60d31ba3eb431bb708868ff16"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump({"username":"shreegowri","key":"f70a15b60d31ba3eb431bb708868ff16"}, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ifood-2019-fgvc6 #downloads the files into /content/{/content}/competitions/ifood-2019-fgvc6 directory

- path is now set to: {/content}
 92% 203M/221M [00:04<00:00, 32.9MB/s]
100% 221M/221M [00:04<00:00, 52.9MB/s]
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 73.3MB/s]
 98% 514M/526M [00:08<00:00, 68.5MB/s]
100% 526M/526M [00:08<00:00, 65.0MB/s]
  0% 0.00/554k [00:00<?, ?B/s]
100% 554k/554k [00:00<00:00, 76.3MB/s]
  0% 0.00/554k [00:00<?, ?B/s]
100% 554k/554k [00:00<00:00, 75.3MB/s]
  0% 0.00/3.63k [00:00<?, ?B/s]
100% 3.63k/3.63k [00:00<00:00, 3.24MB/s]
100% 2.12G/2.12G [00:44<00:00, 73.9MB/s]
100% 2.12G/2.12G [00:44<00:00, 51.1MB/s]
  0% 0.00/217k [00:00<?, ?B/s]
100% 217k/217k [00:00<00:00, 71.0MB/s]


In [0]:
import zipfile

train_zip = '/content/{/content}/competitions/ifood-2019-fgvc6/train_set.zip'
zip_ref = zipfile.ZipFile(train_zip, 'r')
zip_ref.extractall('train_set')


train_label_zip = '/content/{/content}/competitions/ifood-2019-fgvc6/train_labels.csv.zip'
zip_ref = zipfile.ZipFile(train_label_zip, 'r')
zip_ref.extractall('train_set/train_set')


val_zip = '/content/{/content}/competitions/ifood-2019-fgvc6/val_set.zip'
zip_ref = zipfile.ZipFile(val_zip, 'r')
zip_ref.extractall('val_set')

test_zip = '/content/{/content}/competitions/ifood-2019-fgvc6/test_set.zip'
zip_ref = zipfile.ZipFile(test_zip, 'r')
zip_ref.extractall('test_set')

zip_ref.close()

In [0]:
def load_csv(filename):
  ''' param: filename of the csv file with train or test images names along with respective labels
      return: the path to images and respective labels of the images
  '''
  dirname = os.path.dirname(filename)
  images_path = []
  labels = []
  with open(filename) as f:
    parsed = csv.reader(f, delimiter=",", quotechar="'")
    for row in parsed:
      if row[1]=='label':
        continue
      images_path.append(os.path.join(dirname, row[0]))
      #print(row[1])
      labels.append(int(row[1]))
  return images_path, labels
train_images_path, train_labels = load_csv('/content/train_set/train_set/train_labels.csv')
val_images_path, val_labels = load_csv ('/content/val_set/val_set/val_labels.csv')
print(len(train_labels))
class_names = np.loadtxt('/content/class_list.txt', dtype={'names': ('class_num', 'class_name',), 'formats': ('i4', 'U100')})
class_names[0][1]

118475


'macaron'

In [0]:
#input_shape = (3, IMAGE_SIZE, IMAGE_SIZE)
HEIGHT = 299
WIDTH = 299
IMG_SHAPE = (HEIGHT, WIDTH, 3)
resize_height = 299
resize_height = 299
means = [123.68, 116.779, 103.939] #ImageNet's mean, change to training data mean

def preprocess_train(x,y):
  ''' param: path to the image with its label
      task: reads image from directory, resizes an image to 256 x 256, then randomly crops to 224 x 224
      return: the augmented and centered data
  '''
  x = tf.io.read_file(x) #read images
  x = tf.image.decode_jpeg(x, dct_method="INTEGER_ACCURATE", channels=3) #decode jpeg images
  x = tf.image.convert_image_dtype(x, tf.float32)
  x = tf.image.resize(x, (resize_height, resize_height), antialias=False) #Resizes an image to a target width and height by keeping the aspect ratio the same without distortion.
  #x = tf.image.random_crop(x, [HEIGHT, WIDTH, 3]) #Randomly crops a tensor to a given size
  #x = tf.image.central_crop(x, 0.875)  #as we 224/256 = 0.875 central cropping the image
  #x = tf.image.random_flip_left_right(x)  #flips the image along width with 0.5 probability
  x = tf.keras.applications.inception_v3.preprocess_input(x)
  return x,y

In [0]:
def preprocess_val(x,y):
  ''' similar to preprocess_train but uses center crop after resize
  '''
  x = tf.io.read_file(x)
  x = tf.image.decode_jpeg(x, dct_method="INTEGER_ACCURATE", channels=3)
  x = tf.image.convert_image_dtype(x, tf.float32)
  x = tf.image.resize(x, [resize_height, resize_height], antialias=False)
  #x = tf.image.resize_with_pad(x, resize_height, resize_height, antialias=False)
  #x = tf.image.central_crop(x, 0.875)  #as we 224/256 = 0.875 central cropping the image
  x = tf.keras.applications.inception_v3.preprocess_input(x)
  return x,y

In [0]:
#define the TF datset with batchsize, shuffling and preprocessing
BATCH_SIZE = 64 #change and see if performance improves, 1024 resource not enough
num_train_examples = len(train_images_path)
Num_Classes = len(set(train_labels))
train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, train_labels)).shuffle(8000, reshuffle_each_iteration=True).map(preprocess_train, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images_path, val_labels)).map(preprocess_val).batch(64)

In [0]:
base_model = tf.keras.applications.InceptionV3(input_shape=(299,299,3), weights = "imagenet", include_top = False) #input_shape = (224,224,3),
for layer in base_model.layers:
  layer.trainable=True
#last_layer = base_model.get_layer('post_relu')
last_output = base_model.layers[-1].output
x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(last_output)
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation= None, name='dense_layer_1')(x)
x = tf.keras.layers.Lambda(lambda y: tf.math.l2_normalize(y, axis=1))(x)
model = tf.keras.models.Model(base_model.input, x, name='model_embedding')

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
#Adding tensorboard callbacks

from tensorflow.keras.callbacks import TensorBoard
import datetime

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, update_freq='batch', histogram_freq=1)

In [0]:
#Train the model

num_epochs = 10
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), loss = tfa.losses.TripletSemiHardLoss(), metrics=['accuracy'])
#saves the model after every epoch
cp_path = "/content/drive/My Drive/training_val_tp/cp.ckpt" #path to save the model
cp_dir = os.path.dirname(cp_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path, save_weights_only=True,  period=2, verbose=1)
model.save_weights(cp_path.format(epoch=0))
history = model.fit(train_dataset, epochs = num_epochs, validation_data = val_dataset, callbacks = [tensorboard_callback, cp_callback])


In [0]:
%load_ext tensorboard

%tensorboard --logdir logs/fit

In [0]:
# Evaluate the model_1 network
val_results = model2.predict(val_dataset)

val_labels = np.array([]).astype(int)
for element in val_dataset.as_numpy_iterator(): 
  val_labels = np.append(val_labels, element[1])
import io
import tensorflow_datasets as tfds
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", val_results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(val_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()

In [0]:
model_2.load_weights('/content/checkpoints/incp')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f261e6a5ef0> and <tensorflow.python.keras.layers.core.Dropout object at 0x7f261e6a5d30>).


In [0]:
predictions = tf.nn.softmax(model_2.predict(img))

In [0]:
tf.reduce_sum(predictions, axis=1)

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([1.0000001 , 1.0000001 , 0.9999999 , 1.0000001 , 0.9999999 ,
       1.        , 1.        , 0.99999994, 0.9999999 , 0.99999994,
       0.9999999 , 1.0000001 , 0.9999997 , 0.99999976, 1.0000002 ,
       1.        , 0.99999994, 1.        , 1.0000004 , 0.9999996 ,
       1.        , 1.        , 0.9999999 , 1.        , 1.        ,
       1.0000001 , 1.0000001 , 0.9999999 , 1.0000001 , 1.        ,
       1.        , 0.9999998 , 1.0000001 , 0.9999999 , 1.        ,
       1.0000001 , 1.        , 0.9999999 , 1.0000001 , 1.        ,
       1.0000001 , 1.0000001 , 1.0000001 , 1.        , 1.0000001 ,
       1.0000001 , 0.99999994, 0.99999994, 1.0000001 , 1.        ,
       1.0000002 , 0.9999999 , 1.0000001 , 0.9999998 , 1.        ,
       1.0000001 , 1.        , 0.9999998 , 1.        , 1.0000001 ,
       1.0000001 , 1.0000001 , 0.9999997 , 1.0000002 ], dtype=float32)>